In [ ]:
import os
import cv2
import gc
import matplotlib.pyplot    as plt
import numpy                as np
import matplotlib.patches   as mpatches
import plotly.graph_objects as go
import networkx             as nx
import cupy                 as cp
from scipy.stats       import skew, kurtosis, mode
from sklearn.cluster   import DBSCAN
from scipy.ndimage     import convolve, generic_filter
from cupyx.scipy.ndimage import convolve as cpconvolve
from cupyx.scipy.ndimage import binary_dilation
from scipy.signal import convolve2d
from collections import deque
from scipy.interpolate import interp1d

In [ ]:
#point_cloud_indices
name, heights, convolved_heights, intensity, x_to_i, i_to_x, y_to_i, i_to_y, sectioned = 0,1,2,3,4,5,6,7,8

window_height = 15

In [ ]:
#kernels
gauss =  cp.array([
    [1,  4,  7,  4,  1],
    [4, 16, 26, 16,  4],
    [7, 26, 41, 26,  7],
    [4, 16, 26, 16,  4],
    [1,  4,  7,  4,  1]
]) / 273

gauss_y =  cp.array([
    [1],
    [4],
    [7],
    [4],
    [1]
]) / 17
sobel_x = cp.array([
    [ 0.5, 1.25,  0.0,  -1.25, -0.5],
    [   1,  2.5,  0.0,   -2.5,   -1],
    [ 0.5, 1.25,  0.0,  -1.25, -0.5]
]) / 7
simple_x = cp.array([
    [-1,0,1]
]) 

simple_y = cp.array([
    [-1],
    [0],
    [1]
]) 


sobel_x_2 = cp.array([
    [1.25, 0.5, 0.0, -0.5,  -1.25],
    [ 2.5,   1, 0.0,   -1,   -2.5],
    [1.25, 0.5, 0.0, -0.5,  -1.25]
]) / 7

sobel_y = cp.array([
        [-20.75,],
        [ -11.6,],
        [ -6.27,],
        [    -2,],
        [     0,],
        [     2,],
        [  6.27,],
        [  11.6,],
        [ 20.75,]
])
sobel_x_3 = sobel_y.T

In [ ]:
#functions

def get_props_convolved(distribution):
    properties = []

    mean = np.sqrt(np.mean(distribution))
    max = np.sqrt(np.max(distribution))

    variance =  np.var(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    
    properties.append(variance)
    properties.append(skw)
    properties.append(kurt)
    

    norm = np.linalg.norm(properties)
    
    if norm > 0:
        properties = [x / norm for x in properties]
    

    properties.append(mean)
    properties.append(max)

    z = properties[0]
    y = properties[1]
    x = properties[2]

    properties.append((np.arctan2(y,x) + (2 * np.pi)) % (2 * np.pi))
    properties.append(np.arctan(z)/(np.pi/2))


    return properties

def get_props(distribution):
    properties = []

    variance =  np.var(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    
    properties.append(variance)
    properties.append(skw)
    properties.append(kurt)

    norm = np.linalg.norm(properties)
    
    if norm > 0:
        properties = [x / norm for x in properties]

    z = properties[0]

    properties.append(np.arccos(z)/(np.pi/2))

    return properties

def classify(distribution, distribution2):
    e = 3
    v = get_props(distribution)
    if (1 - v[3]) >= np.exp(-e*v[1]):
        return 1 
    elif (1 - v[3]) >= np.exp(e*v[1]):
        return 2
    else:
        return 3

def get_point_cloud(paths, y_window = window_height + 5, upsample_ratio = 2):
    print("loading",end = "... ")
    with open(paths[1]) as file:
            lines = file.readlines()
            for line in lines:
                if "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                    x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
                if "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                    x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())

    with open(paths[2]) as file:
        lines = file.readlines()
        transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

    imread = lambda fn: cv2.imread(fn, cv2.IMREAD_ANYDEPTH)
    
    point_cloud  = np.fromfile(paths[3], dtype=np.float32).reshape(-1, 3) 
    intensity_map = imread(paths[4])
    
    print(f"{paths[0]} loaded, {point_cloud.shape[0]} points")
    print("trimming",end = "... ")
    
    intensity_values = np.reshape(intensity_map, (-1, 1))
    intensity_cloud = np.hstack((point_cloud[:,:2], intensity_values))

    point_cloud = (np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1)))) @ transformation_matrix.T)[:,:3]
    intensity_cloud = (np.hstack((intensity_cloud, np.ones((intensity_cloud.shape[0], 1)))) @ transformation_matrix.T)[:,:3]

    mask = (point_cloud[:,0] <= x_start) & (point_cloud[:,0] >= x_stop) & (np.abs(point_cloud[:,1]) <= y_window)
    point_cloud = point_cloud[mask]
    intensity_cloud = intensity_cloud[mask]

    min_intensity = np.nanmax(intensity_cloud[:,2])
    min_z = np.nanmax(point_cloud[:,2])
    
    print(f"trimmed to {point_cloud.shape[0]} points")

    print("converting to arrays",end = "... ")

    minimum_x = point_cloud[np.argmin(np.abs(point_cloud[:,0] - x_stop)),0]

    point_cloud[:,0] -= minimum_x
    intensity_cloud[:,0] -= minimum_x
    
    x_values = np.unique(point_cloud[:,0])
    y_values = np.unique(point_cloud[:,1])

    index_step = (np.nanmedian(np.diff(x_values))) / upsample_ratio 
    index_steps = np.arange(int(round(np.max(x_values)/index_step))+1) * index_step

    x_range = len(index_steps)
    y_value_dict = {y: index for index,y in enumerate(y_values)}
    x_value_dict = {x: np.argmin(np.abs(index_steps - x)) for x in x_values}
    known_indices = list(x_value_dict.values())
    known_x_values = np.array(list(x_value_dict.keys()), dtype=float)
    all_indices = np.arange(x_range)
    
    interp_func = interp1d(known_indices, known_x_values, kind="linear", fill_value="extrapolate")
    x_values = interp_func(all_indices)
    
    x_value_dict = {x: index for index,x in enumerate(x_values)}
    
    point_array = np.full((len(y_values), x_range),np.nan)
    intensity_array = np.full((len(y_values), x_range),np.nan)
    point_dictionary = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(point_cloud)}
    intensity_dictionary = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(intensity_cloud)}

    for x in range (x_range):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]

            point_z = point_dictionary.get((x_val,y_val))
            intensity_z = intensity_dictionary.get((x_val,y_val))

            if point_z is not None:
                point_array[y,x] = point_z[1]
            else:
                point_array[y,x] = min_z
            if intensity_z is not None:
                intensity_array[y,x] = intensity_z[1]
            else:
                intensity_array[y,x] = min_intensity

    print(f"arrays built")
    print("upsampling",end = "... ")

    for y in range(len(y_values)):
        interpolated_z_values= []
        interpolated_i_values = []

        for dy in [-1,0,1]:
            y_dy = y + dy
            if (y_dy >= 0) & (y_dy < len(y_values)):
                known_z = point_array[y_dy,known_indices]
                global known_x
                known_i = intensity_array[y_dy,known_indices]
                known_x = x_values[known_indices]
                mask = ~np.isnan(known_z)
                z_interp = interp1d(known_x[mask], known_z[mask], kind='linear', fill_value="extrapolate")
                i_interp = interp1d(known_x[mask], known_i[mask], kind='linear', fill_value="extrapolate")
    
                interpolated_z_values.append(z_interp(x_values))
                interpolated_i_values.append(i_interp(x_values))
        
        point_array[y, :] = np.mean(interpolated_z_values, axis=0) 
        intensity_array[y, :] = np.mean(interpolated_i_values, axis=0) 
    
    intensity_array = np.log(np.abs(intensity_array))
    kernel_y = np.array([[-1],[-2],[0],[2],[1]])
    convolved_array = np.abs(convolve(point_array,kernel_y))

    print(f"upsampled to {point_array.size} points")
    print(f"{paths[0]} finished \n")

    return [paths[0],point_array, convolved_array, intensity_array, x_value_dict, x_values, y_value_dict, y_values]

def get_gradient(data):
    array_i = cp.array(data[intensity], dtype=cp.float32)
    array_z = cp.array(data[heights], dtype=cp.float32)
    y_values = data[i_to_y]

    y_grad_i = cp.abs(cpconvolve(array_i, sobel_y))
    x_grad_i = cp.abs(cpconvolve(array_i, sobel_x_3))
    magnitude_i = cp.sqrt(x_grad_i**2 + y_grad_i**2)
    y_grad_i = cp.abs(cpconvolve(array_i, sobel_y))
    magnitude_i = cp.sqrt(x_grad_i**2 + y_grad_i**2)

    x_grad_z = cp.abs(cpconvolve(array_z, sobel_y))
    y_grad_z = cp.abs(cpconvolve(array_z, sobel_x_3))
    magnitude_z = cp.sqrt(x_grad_z**2 + y_grad_z**2)
    y_grad_z = cp.abs(cpconvolve(magnitude_z, sobel_x_3))
    magnitude_z = cp.sqrt(x_grad_z**2 + y_grad_z**2)


    magnitude = np.sqrt(magnitude_i * (magnitude_z/5))

    y_bottom = np.argmin(np.abs(y_values - window_height))
    y_top = np.argmin(np.abs(y_values + window_height))

    magnitude[:y_top, :] = 0
    magnitude[y_bottom:, :] = 0
    
    return cp.asnumpy(magnitude)

def PCA(data, y_window=5, x_window=4, s = 0.001, w = 0.0005):
    array = cp.array(data[heights], dtype=cp.float32)
    y_values = data[i_to_y]
    height, width = array.shape
    y_range, x_range = 2 * y_window + 1, 2 * x_window + 1

    y_bottom = np.argmin(np.abs(y_values - window_height))
    y_top = np.argmin(np.abs(y_values + window_height))

    x_n = cp.arange(-x_window, x_window + 1) /2
    y_n = cp.arange(-y_window, y_window + 1) /4.5
    valid_x = cp.repeat(x_n, y_range).flatten()
    valid_y = cp.tile(y_n, x_range).flatten()

    points = cp.lib.stride_tricks.sliding_window_view(
        cp.pad(array, ((y_window,y_window), (x_window,x_window)), mode='edge'), (y_range, x_range)
    )
    del array

    points = points.reshape(height,width, y_range*x_range)
    
    points = cp.stack((
        cp.broadcast_to(valid_x, (height,width, y_range*x_range)),
        cp.broadcast_to(valid_y, (height,width, y_range*x_range)),
        points), axis=2)
    mean_vals = cp.mean(points[:,:,2,:], axis=(2), keepdims=True)
    points[:,:,2,:] -= mean_vals

    cov_matrices = cp.matmul(points, points.transpose(0, 1, 3, 2)) / (points.shape[3] - 1)
    del points 

    a, b, c, d, e, f, g, h, i = (
        cov_matrices[:, :, 0, 0], cov_matrices[:, :, 0, 1], cov_matrices[:, :, 0, 2],
        cov_matrices[:, :, 1, 0], cov_matrices[:, :, 1, 1], cov_matrices[:, :, 1, 2],
        cov_matrices[:, :, 2, 0], cov_matrices[:, :, 2, 1], cov_matrices[:, :, 2, 2]
    )

    p1 = b**2 + c**2 + f**2
    q = (a + e + i) / 3
    p2 = (a - q)**2 + (e - q)**2 + (i - q)**2 + 2 * p1
    p = cp.sqrt(p2 / 6)
    
    B = (cov_matrices - cp.eye(3, dtype=cp.float32) * q[:, :, None, None]) / p[:, :, None, None]
    r = cp.linalg.det(B) / 2
    phi = cp.arccos(cp.clip(r, -1, 1)) / 3
    del cov_matrices

    eigvals_2 = q + 2 * p * cp.cos(phi + (2 * cp.pi / 3))
    eigvals_2 = cp.asnumpy(eigvals_2)
    eigvals_2[:y_top, :] = 0
    eigvals_2[y_bottom:, :] = 0

    neighbours = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    strong_y, strong_x = np.where(eigvals_2 >= s)
    weak_values = (eigvals_2 >= w) & (eigvals_2 < s).astype(np.uint8) 
    return_array = np.zeros_like(eigvals_2, dtype=cp.uint8)
    
    cp.get_default_memory_pool().free_all_blocks()

    queue = deque(zip(strong_y, strong_x))
    while queue:
        y, x = queue.popleft() 
        return_array[y, x] = 1
        for dy, dx in neighbours:
            ny, nx = y + dy, x + dx
            if 0 <= ny < weak_values.shape[0] and 0 <= nx < weak_values.shape[1]:
                if weak_values[ny, nx]:
                    weak_values[ny, nx] = 0 
                    queue.append((ny, nx)) 
    del weak_values 

    return return_array

def get_edges(data,s = 50, w = 15, y_window=9, x_window=2, s_e = 0.0125, w_e = 0.0075):
    mask = PCA(data, y_window, x_window, s = s_e, w = w_e)
    NMS_magnitude = get_gradient(data)

    y_values = data[i_to_y]
    neighbours = [(-1, 0), (0, -1), (0, 1), (1, 0)]
   
    return_array = np.zeros_like(NMS_magnitude)

    NMS_array_masked = NMS_magnitude * mask 
    
    strong_y, strong_x = np.where(NMS_array_masked >= s)
    weak_edges = (NMS_magnitude >= w) & (NMS_array_masked < s).astype(np.uint8) 

    queue = deque(zip(strong_y, strong_x))
    while queue:
        y, x = queue.popleft() 
        return_array[y, x] = 1
        for dy, dx in neighbours:
            ny, nx = y + dy, x + dx
            if 0 <= ny < mask.shape[0] and 0 <= nx < mask.shape[1]:
                if weak_edges[ny, nx]:
                    weak_edges[ny, nx] = 0 
                    queue.append((ny, nx)) 

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,18))
    y_top = np.argmin(np.abs(y_values - window_height))
    y_bottom = np.argmin(np.abs(y_values + window_height))
    y_seed_point = np.argmin(np.abs(y_values))

    closed = cv2.morphologyEx(return_array, cv2.MORPH_CLOSE, kernel)
    closed[y_top, :] = 1
    closed[y_bottom, :] = 1

    return np.where(closed ==  1, 1 , np.nan), y_seed_point
    
def draw_sections(edge_array):
    edges1, seed_index = edge_array
    edges = edges1.copy() 

    neighbours = [(-1, 0), (0, -1), (0, 1), (1, 0)]  
    
    nan_mask = np.isnan(edges)  
    int = 2

    for x in range(edges.shape[1]):

        if nan_mask[seed_index, x]: 
            queue = deque([(seed_index, x)])

            while queue:
                y, x = queue.popleft()
                edges[y, x] = int 
                nan_mask[y, x] = False  
                
                neighbors_to_add = [
                    (ny, nx) for dy, dx in neighbours
                    if (0 <= (ny := y + dy) < edges.shape[0] and 
                        0 <= (nx := x + dx) < edges.shape[1] and 
                        nan_mask[ny, nx])  
                ]

                queue.extend(neighbors_to_add) 
                for ny, nx in neighbors_to_add:
                    nan_mask[ny, nx] = False 
                    
        int += 1
    edges = np.where(edges > 1, edges, np.nan)

    return edges

def define_sections(data):
    edges, seed_index = get_edges(data)
    z_array = data[heights]
    neighbours = [(-1, 0), (0, -1), (0, 1), (1, 0)]  
    distributions = []
    x_values = []
    nan_mask = np.isnan(edges) 

    print("defining sections...")

    for x in range(edges.shape[1]):
        center_x_vals = []
        avg_z_vals = {}
        size = 0
        if nan_mask[seed_index, x]: 
            queue = deque([(seed_index, x)])
            while queue:
                y, x = queue.popleft()
                size += 1

                if y == seed_index:
                    center_x_vals.append(x)

                if y not in avg_z_vals.keys():
                    avg_z_vals[y] = []

                avg_z_vals[y].append(z_array[y,x])
                nan_mask[y, x] = False  

                neighbors_to_add = [
                    (ny, nx) for dy, dx in neighbours
                    if (0 <= (ny := y + dy) < edges.shape[0] and 
                        0 <= (nx := x + dx) < edges.shape[1] and 
                        nan_mask[ny, nx])  
                ]
                queue.extend(neighbors_to_add) 
                for ny, nx in neighbors_to_add:
                    nan_mask[ny, nx] = False
        
        if size > 1000:
            avg_z_vals = [np.nanmedian(z_values) for z_values in avg_z_vals.values()]
            distributions.append(avg_z_vals)
            x_values.append(center_x_vals)
    


    return distributions,x_values

def label_x_values(data):
    print("finding edges...")
    edges, seed_index = get_edges(data)
    z_array = data[heights]
    cz_array = data[convolved_heights]
    neighbours = [(-1, 0), (0, -1), (0, 1), (1, 0)]
    x_values = {x: 0 for x in range (z_array.shape[1])}
    nan_mask = np.isnan(edges) 

    print("classifying sections...")

    for x in range(edges.shape[1]):
        center_x_vals = []
        avg_z_vals = {}
        avg_cz_vals = {}
        size = 0
        if nan_mask[seed_index, x]: 
            queue = deque([(seed_index, x)])
            while queue:
                y, x = queue.popleft()
                size += 1

                if y == seed_index:
                    center_x_vals.append(x)

                if y not in avg_z_vals.keys():
                    avg_z_vals[y] = []
                    avg_cz_vals[y] = []

                avg_z_vals[y].append(z_array[y,x])
                avg_cz_vals[y].append(cz_array[y,x])
                nan_mask[y, x] = False  

                neighbors_to_add = [
                    (ny, nx) for dy, dx in neighbours
                    if (0 <= (ny := y + dy) < edges.shape[0] and 
                        0 <= (nx := x + dx) < edges.shape[1] and 
                        nan_mask[ny, nx])  
                ]
                queue.extend(neighbors_to_add) 
                for ny, nx in neighbors_to_add:
                    nan_mask[ny, nx] = False
        
        if size > 1000:
            avg_z_vals = [np.nanmedian(z_values) for z_values in avg_z_vals.values()]
            avg_cz_vals = [np.nanmedian(z_values) for z_values in avg_cz_vals.values()]
            label = classify(avg_z_vals,avg_cz_vals)
            for x in center_x_vals:
                x_values[x] = label
    
    return x_values

In [ ]:
class FileParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                    if dir_name.startswith("Box"):
                        adaptive_z_folder = os.path.join(root, dir_name, "AdaptiveZ_10mm")

                        if os.path.isdir(adaptive_z_folder): 
                            for part_folder in os.listdir(adaptive_z_folder):
                                full_part_path = os.path.join(adaptive_z_folder, part_folder)
                                if os.path.isdir(full_part_path) and "Part" in part_folder:
                                    component_parameters_path = None
                                    lidar2xrf_path = None
                                    bpc_path = None
                                    intensity_path = None
                                    for file_name in os.listdir(full_part_path):
                                        if file_name.endswith(".component_parameters.txt"):
                                            component_parameters_path = os.path.join(full_part_path, file_name)
                                        elif file_name.endswith(".lidar2xrf"):
                                            lidar2xrf_path = os.path.join(full_part_path, file_name)
                                        elif file_name.endswith(".bpc"):
                                            bpc_path = os.path.join(full_part_path, file_name)
                                        elif file_name.endswith("_intensity.png"):
                                            intensity_path = os.path.join(full_part_path, file_name)

                                    self.box_folders.append((part_folder,component_parameters_path, lidar2xrf_path, bpc_path, intensity_path))

    def get_box_folders(self):
        self.find_folders()
        return self.box_folders




parser = FileParser(r"C:\Users\eashenhurst\Desktop\local macassa")
paths_list = parser.get_box_folders()

for title, component_parameters_path, lidar2xrf_path, bpc_path, intensity_path in paths_list:
    print(f"Part: {title}")
    if component_parameters_path:
        print(f"  Component Parameters: {component_parameters_path}")
    if lidar2xrf_path:
        print(f"  LIDAR to XRF: {lidar2xrf_path}")
    if bpc_path:
        print(f"  BPC File: {bpc_path}")
    if intensity_path:
        print(f"  Intensity File: {intensity_path}")


In [ ]:
class auzzyFileParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                    if dir_name.startswith("Box"):
                        folder = os.path.join(root, dir_name)

                        if os.path.isdir(folder): 
                            full_part_path = os.path.join(folder, folder)
                            if os.path.isdir(full_part_path) and "Part" in folder:
                                component_parameters_path = None
                                lidar2xrf_path = None
                                bpc_path = None
                                for file_name in os.listdir(full_part_path):
                                    if file_name.endswith(".component_parameters.txt"):
                                        component_parameters_path = os.path.join(full_part_path, file_name)
                                    elif file_name.endswith(".lidar2xrf"):
                                        lidar2xrf_path = os.path.join(full_part_path, file_name)
                                    elif file_name.endswith(".bpc"):
                                        bpc_path = os.path.join(full_part_path, file_name)
                                    elif file_name.endswith("_intensity.png"):
                                        intensity_path = os.path.join(full_part_path, file_name)
                                self.box_folders.append((folder,component_parameters_path, lidar2xrf_path, bpc_path, intensity_path))

    def get_box_folders(self):
        self.find_folders()
        return self.box_folders


parser = auzzyFileParser(r"C:\Users\eashenhurst\Desktop\auzzy rocks")

paths_list = parser.get_box_folders()


for title, component_parameters_path, lidar2xrf_path, bpc_path, intensity_path in paths_list:
    print(f"Part: {title}")
    print(f"  Component Parameters: {component_parameters_path}")
    print(f"  LIDAR to XRF: {lidar2xrf_path}")
    print(f"  BPC File: {bpc_path}")
    print(f"  Intensity File: {intensity_path}")

In [ ]:
#Get the point clouds
point_clouds = []

for paths in paths_list: 
    point_clouds.append(get_point_cloud(paths))

In [ ]:
for point_cloud in point_clouds:
    if len(point_cloud) > sectioned:
        point_cloud[sectioned] = label_x_values(point_cloud)
    else:
        point_cloud.append(label_x_values(point_cloud))
    print(f"done {point_cloud[0]} .\n")

In [ ]:
fig = plt.figure(figsize=(70, 3 * len(point_clouds)), dpi=75)
gs = fig.add_gridspec(len(point_clouds), 1, hspace=0.025)

color_map = np.array(["black", "red", "cyan", "green"])  



for i, pc in enumerate(point_clouds)
    middle_y = np.argmin(np.abs(point_cloud[i_to_y]))
    ax = fig.add_subplot(gs[i, 0])
    display = pc[intensity]  
    ax.imshow(np.flipud(display), cmap='bone', interpolation='nearest', alpha=1)

    x_vals = np.arange(display.shape[1])
    labels = np.array(list(pc[sectioned].values()))  

    x_vals = x_vals[::4]
    labels = labels[::4]

    colors = color_map[labels]  
    
    ax.bar(x_vals, height = 4, width= 4, bottom = middle_y - 2, color = colors, alpha = 0.65)

    ax.set_xlabel("X index")
    ax.set_ylabel("Y index")
    ax.set_title(pc[name], fontsize=20)

plt.show()

In [ ]:
section_displays = []

for point_cloud in point_clouds:
    edges = get_edges(point_cloud,s = 50, w = 15, y_window=9, x_window=2, s_e = 0.0125, w_e = 0.0075)
    sections = draw_sections(edges)
    section_displays.append(sections)
    print(f"done {point_cloud[0]}")

In [ ]:
fig = plt.figure(figsize=(70, 3*len(point_clouds)), dpi=100)
gs = fig.add_gridspec(len(point_clouds), 1, hspace=0.025)


for (i,pc) , (i2, section_array) in zip(enumerate(point_clouds), enumerate(section_displays)):
   ax = fig.add_subplot(gs[i, 0])
   
   ax.imshow(cp.flipud(pc[heights]), cmap='bone_r', interpolation='nearest', alpha = 1) 
   ax.imshow(cp.flipud(pc[intensity]), cmap='bone', interpolation='nearest', alpha = 0.65) 
   ax.imshow(cp.flipud(section_array), cmap='nipy_spectral', interpolation='nearest', alpha = 0.65) 
   ax.set_xlabel("X index")
   ax.set_ylabel("Y index")
   ax.set_title(pc[name],fontsize = 20)
